# Recommandation Netflix user

We would like you to create some movie recommendations for our very special user, Monica Villarreal. Can you please recommend 5 other movies to her? We would like Monica to watch more movies on our website so that she remains a loyal customer. 

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
data = pd.read_csv("Netmovies_ratings_db.csv")

user_item_matrix = data.pivot_table(
    index="userName", 
    columns="title", 
    values="rating"
)
user="Monica Villarreal"


def get_similar_users(target_user, n=5):
    if target_user not in user_item_matrix.index:
        return f"User '{target_user}' not found."

    similarities = {}
    for other_user in user_item_matrix.index:
        if other_user == target_user:
            continue
        #selected only the movies rated by both users
        common_movies = user_item_matrix.loc[target_user].dropna().index.intersection(
                        user_item_matrix.loc[other_user].dropna().index)
        if len(common_movies) == 0:
            continue
        v1 = user_item_matrix.loc[target_user, common_movies].to_numpy()
        v2 = user_item_matrix.loc[other_user, common_movies].to_numpy()
        #cosine similarity calculation
        sim = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        similarities[other_user] = sim

    
    similar_users = sorted(similarities, key=similarities.get, reverse=True)[:n]
    return similar_users
similar_users = get_similar_users(user)
print("Similar users to", user, ":", get_similar_users(user))

Similar users to Monica Villarreal : ['Caitlyn Hernandez', 'Kathleen Parker', 'Aaron Flores', 'Aaron Thomas', 'Alan Holland']


With the n most close user we can select the recommended films. 

We will chose the five more viewed films by the closet person

In [8]:
def get_proposed_films(user):
    """
    Get a list of films rated by the user.
    
    """
    list_film_monica=data[data['userName']==user]['title'].tolist()
    dict_film_proposed={}
    for user in similar_users:
        list_film_user=data[data['userName']==user]['title'].tolist()
        for film in list_film_user:
            if film not in list_film_monica:
                if film not in dict_film_proposed:
                    dict_film_proposed[film]=1
                else:
                    dict_film_proposed[film]+=1

    sorted_dict_film_proposed=sorted(dict_film_proposed.items(), key=lambda x: x[1], reverse=True)

    return sorted_dict_film_proposed

print("Recommended films:")
for film in get_proposed_films(user)[:5]:
    print(film)


Recommended films:
('Batman Returns', 4)
('Silent Hill', 4)
('To Kill a Mockingbird', 3)
('5 Card Stud', 3)
('Boogie Nights', 3)
